In [1]:
import warnings
warnings.filterwarnings("ignore")
from copy import deepcopy
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import sys

sys.path.append("/code")

from tqdm import tqdm
import torch
# device = torch.device('cpu')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# import gym
# import recogym

import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim

torch.backends.cudnn.benchmark = torch.cuda.is_available()
if torch.cuda.is_available():
    torch.set_float32_matmul_precision("high")  # TF32 = big speedup on Ada


from sklearn.utils import check_random_state

# implementing OPE of the IPWLearner using synthetic bandit data
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt

from scipy.special import softmax
import optuna
# from memory_profiler import profile


from estimators import (
    DirectMethod as DM
)

from simulation_utils import (
    eval_policy,
    generate_dataset,
    create_simulation_data_from_pi,
    get_train_data,
    get_opl_results_dict,
    CustomCFDataset,
    calc_reward,
    get_weights_info
)

from models import (    
    LinearCFModel,
    NeighborhoodModel,
    BPRModel, 
    RegressionModel
)

from training_utils import (
    train,
    validation_loop, 
    cv_score_model
 )

from custom_losses import (
    SNDRPolicyLoss,
    IPWPolicyLoss, 
    KLPolicyLoss
    )

random_state=12345
random_ = check_random_state(random_state)

pd.options.display.float_format = '{:,.8f}'.format

Using device: cuda
Using device: cuda
Using device: cuda


In [2]:
def get_trial_results(
    our_x, 
    our_a, 
    emb_x, 
    emb_a, 
    original_x, 
    original_a, 
    dataset, 
    val_data, 
    original_policy_prob, 
    neighberhoodmodel, 
    regression_model, 
    dm
):
    policy = np.expand_dims(softmax(our_x @ our_a.T, axis=1), -1)
    policy_reward = calc_reward(dataset, policy)
    eval_metrics = eval_policy(neighberhoodmodel, val_data, original_policy_prob, policy)
    action_diff_to_real = np.sqrt(np.mean((emb_a - our_a) ** 2))
    action_delta = np.sqrt(np.mean((original_a - our_a) ** 2))
    context_diff_to_real = np.sqrt(np.mean((emb_x - our_x) ** 2))
    context_delta = np.sqrt(np.mean((original_x - our_x) ** 2))

    row = np.concatenate([
        np.atleast_1d(policy_reward),
        np.atleast_1d(eval_metrics),
        np.atleast_1d(action_diff_to_real),
        np.atleast_1d(action_delta),
        np.atleast_1d(context_diff_to_real),
        np.atleast_1d(context_delta)
    ])
    reg_dm = dm.estimate_policy_value(policy[val_data['x_idx']], regression_model.predict(val_data['x']))
    reg_results = np.array([reg_dm])
    conv_results = np.array([row])
    return get_opl_results_dict(reg_results, conv_results)

## `trainer_trial` Function

This function runs policy learning experiments using offline bandit data and evaluates various estimators.

### Parameters
- **num_runs** (int): Number of experimental runs per training size
- **num_neighbors** (int): Number of neighbors to consider in the neighborhood model
- **num_rounds_list** (list): List of training set sizes to evaluate
- **dataset** (dict): Contains dataset information including embeddings, action probabilities, and reward probabilities
- **batch_size** (int): Batch size for training the policy model
- **num_epochs** (int): Number of training epochs for each experiment
- **lr** (float, default=0.001): Learning rate for the optimizer

### Process Flow
1. Initializes result structures and retrieval models
2. For each training size in `num_rounds_list`:
   - Creates a uniform logging policy and simulates data
   - Generates training data for offline learning
   - Fits regression and neighborhood models for reward estimation
   - Initializes and trains a counterfactual policy model
   - Evaluates policy performance using various estimators
   - Collects metrics on policy reward and embedding quality

### Returns
- **DataFrame**: Results table with rows indexed by training size and columns for various metrics:
  - `policy_rewards`: True expected reward of the learned policy
  - Various estimator errors (`ipw`, `reg_dm`, `conv_dm`, `conv_dr`, `conv_sndr`)
  - Variance metrics for each estimator
  - Embedding quality metrics comparing learned representations to ground truth

### Implementation Notes
- Uses uniform random logging policy for collecting offline data
- Employs Self-Normalized Doubly Robust (SNDR) policy learning
- Measures embedding quality via RMSE to original/ground truth embeddings

In [3]:
def trainer_trial(
    num_runs,
    num_neighbors,
    train_sizes,
    dataset,
    batch_size,
    val_size=2000,
    n_trials=10,    
    prev_best_params=None
):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    torch.backends.cudnn.benchmark = torch.cuda.is_available()
    if torch.cuda.is_available():
        torch.set_float32_matmul_precision("high")

    dm = DM()
    results = {}

    our_x, our_a = dataset["our_x"], dataset["our_a"]
    emb_x, emb_a = dataset["emb_x"], dataset["emb_a"]

    original_x, original_a = dataset["original_x"], dataset["original_a"]
    n_users, n_actions, emb_dim = dataset["n_users"], dataset["n_actions"], dataset["emb_dim"]

    all_user_indices = np.arange(n_users, dtype=np.int64)

    def T(x):
        return torch.as_tensor(x, device=device, dtype=torch.float32)

    def _mean_dict(dicts):
        """
        Robust mean over a list of dicts with numeric/scalar/1D-array values.
        Returns a single dict with elementwise means.
        """
        if not dicts:
            return {}
        keys = dicts[0].keys()
        out = {}
        for k in keys:
            vals = [d[k] for d in dicts if k in d]
            # try to convert each to np.array and average
            arrs = [np.asarray(v) for v in vals]
            # broadcast to same shape if scalars/1D
            stacked = np.stack(arrs, axis=0)
            out[k] = np.mean(stacked, axis=0)
        return out

    # ===== unpack dataset (keep originals safe) =====
    our_x_orig, our_a_orig = our_x, our_a
    emb_x, emb_a = emb_x, emb_a
    original_x, original_a = original_x, original_a
    n_users, n_actions, emb_dim = n_users, n_actions, emb_dim
    all_user_indices = np.arange(n_users, dtype=np.int64)

    dm = DM()
    results = {}
    best_hyperparams_by_size = {}
    last_best_params = prev_best_params if prev_best_params is not None else None

    # ===== baseline (sample size = 0) using get_trial_results =====
    pi_0 = softmax(our_x_orig @ our_a_orig.T, axis=1)
    original_policy_prob = np.expand_dims(pi_0, -1)

    simulation_data = create_simulation_data_from_pi(
        dataset, pi_0, val_size, random_state=0
    )

    # use same data for train/val just to generate the baseline row
    train_data = get_train_data(n_actions, val_size, simulation_data, np.arange(val_size), our_x_orig)
    val_data   = get_train_data(n_actions, val_size, simulation_data, np.arange(val_size), our_x_orig)

    regression_model = RegressionModel(
        n_actions=n_actions, action_context=our_x_orig,
        base_model=LogisticRegression(random_state=12345)
    )

    regression_model.fit(train_data['x'], train_data['a'], train_data['r'])

    neighberhoodmodel = NeighborhoodModel(
        train_data['x_idx'], train_data['a'],
        our_a_orig, our_x_orig, train_data['r'],
        num_neighbors=num_neighbors
    )

    # baseline row produced via get_trial_results
    results[0] = get_trial_results(
        our_x_orig, our_a_orig, emb_x, emb_a, original_x, original_a,
        dataset, val_data, original_policy_prob,
        neighberhoodmodel, regression_model, dm
    )

    # ===== main loop over training sizes =====
    for train_size in train_sizes:

        # we’ll collect per-run trial dicts generated by get_trial_results
        trial_dicts_this_size = []
        best_hyperparams_by_size[train_size] = {}

        # --- prepare a resampling for Optuna’s objective (shared loaders built per-run inside objective) ---
        # We’ll do Optuna per-run (fresh resample + search), then final fit with best params, then get_trial_results.

        for run in range(num_runs):

            # --- resample for this run ---
            pi_0 = softmax(our_x_orig @ our_a_orig.T, axis=1)
            original_policy_prob = np.expand_dims(pi_0, -1)

            simulation_data = create_simulation_data_from_pi(
                dataset, pi_0, train_size + val_size,
                random_state=(run + 1) * (train_size + 17)
            )

            idx_train = np.arange(train_size)
            train_data = get_train_data(n_actions, train_size, simulation_data, idx_train, our_x_orig)
            val_idx   = np.arange(val_size) + train_size
            val_data  = get_train_data(n_actions, val_size, simulation_data, val_idx, our_x_orig)

            num_workers = 4 if torch.cuda.is_available() else 0

            cf_dataset = CustomCFDataset(
                train_data['x_idx'], train_data['a'], train_data['r'], original_policy_prob
            )

            # val_loader = DataLoader(
            #     val_dataset, batch_size=val_size, shuffle=False,
            #     pin_memory=torch.cuda.is_available(),
            #     num_workers=num_workers, persistent_workers=bool(num_workers)
            # )


            # --- Optuna objective bound to this run's data ---
            def objective(trial):
                print()
                print(f"Trial {trial.number} started")
                lr = trial.suggest_float("lr", 1e-4, 1e-1, log=True)
                epochs = trial.suggest_int("num_epochs", 1, 10)
                trial_batch_size = trial.suggest_categorical("batch_size", [64, 128, 256, 512])
                trial_num_neighbors = trial.suggest_int("num_neighbors", 3, 15)
                lr_decay = trial.suggest_float("lr_decay", 0.8, 1.0)

                trial_neigh_model = NeighborhoodModel(
                    train_data['x_idx'], train_data['a'],
                    our_a_orig, our_x_orig, train_data['r'],
                    num_neighbors=trial_num_neighbors
                )

                trial_scores_all = torch.as_tensor(
                    trial_neigh_model.predict(all_user_indices),
                    device=device, dtype=torch.float32
                )

                trial_model = LinearCFModel(
                    n_users, n_actions, emb_dim,
                    initial_user_embeddings=T(our_x_orig),
                    initial_actions_embeddings=T(our_a_orig)
                ).to(device)

                assert (not torch.cuda.is_available()) or next(trial_model.parameters()).is_cuda

                final_train_loader = DataLoader(
                    cf_dataset, batch_size=trial_batch_size, shuffle=True,
                    pin_memory=torch.cuda.is_available(),
                    num_workers=num_workers, persistent_workers=bool(num_workers)
                )

                current_lr = lr
                for epoch in range(epochs):
                    if epoch > 0:
                        current_lr *= lr_decay
                        
                    train(
                        trial_model, final_train_loader, trial_scores_all,
                        criterion=KLPolicyLoss(), num_epochs=1, lr=current_lr, device=str(device)
                    )

                trial_x, trial_a = trial_model.get_params()
                trial_x = trial_x.detach().cpu().numpy()
                trial_a = trial_a.detach().cpu().numpy()

                pi_i = softmax(trial_x @ trial_a.T, axis=1)
                train_actions = train_data['a']
                train_users = train_data['x_idx']

                print("Train wi info: {}".format(get_weights_info(pi_i[train_users, train_actions], original_policy_prob[train_users, train_actions])))
                print(f"actual reward: {calc_reward(dataset, np.expand_dims(pi_i, -1))}")

                # print(get_weights_info(pi_i, original_policy_prob))
                # validation reward for selection
                return cv_score_model(val_data, trial_scores_all, pi_i)


            # --- run Optuna for this run ---
            study = optuna.create_study(direction="maximize")
            
            if last_best_params is not None:
                study.enqueue_trial(last_best_params)

            study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

            best_params = study.best_params
            last_best_params = best_params  # optional warm-start to next run
            best_hyperparams_by_size[train_size][run] = {
                "params": best_params,
                "reward": study.best_value
            }


            # --- final training with best params on this run’s data ---
            regression_model = RegressionModel(
                n_actions=n_actions, action_context=our_x_orig,
                base_model=LogisticRegression(random_state=12345)
            )
            regression_model.fit(
                train_data['x'], train_data['a'], train_data['r'],
                original_policy_prob[train_data['x_idx'], train_data['a']].squeeze()
            )

            neighberhoodmodel = NeighborhoodModel(
                train_data['x_idx'], train_data['a'],
                our_a_orig, our_x_orig, train_data['r'],
                num_neighbors=best_params['num_neighbors']
            )
            scores_all = torch.as_tensor(
                neighberhoodmodel.predict(all_user_indices),
                device=device, dtype=torch.float32
            )

            model = LinearCFModel(
                n_users, n_actions, emb_dim,
                initial_user_embeddings=T(our_x_orig),
                initial_actions_embeddings=T(our_a_orig)
            ).to(device)
            assert (not torch.cuda.is_available()) or next(model.parameters()).is_cuda

            train_loader = DataLoader(
                cf_dataset, batch_size=batch_size, shuffle=True,
                pin_memory=torch.cuda.is_available(),
                num_workers=num_workers, persistent_workers=bool(num_workers)
            )

            current_lr = best_params['lr']
            for epoch in range(best_params['num_epochs']):
                if epoch > 0:
                    current_lr *= best_params['lr_decay']
                train(
                    model, train_loader, scores_all,
                    criterion=KLPolicyLoss(), num_epochs=1, lr=current_lr, device=str(device)
                )

            # learned embeddings (do NOT overwrite originals)
            learned_x_t, learned_a_t = model.get_params()
            learned_x = learned_x_t.detach().cpu().numpy()
            learned_a = learned_a_t.detach().cpu().numpy()

            # --- produce the per-run result via get_trial_results ---
            trial_res = get_trial_results(
                learned_x, learned_a,          # learned (policy) embeddings
                emb_x, emb_a,                  # ground-truth embedding refs
                original_x, original_a,        # original clean refs
                dataset,
                val_data,                      # use this run's val split
                original_policy_prob,
                neighberhoodmodel,
                regression_model,
                dm
            )

            trial_dicts_this_size.append(trial_res)

            # memory hygiene
            torch.cuda.empty_cache()

        # === aggregate per-run results (mean) and store under this train_size ===
        results[train_size] = _mean_dict(trial_dicts_this_size)

    return pd.DataFrame.from_dict(results, orient='index'), best_hyperparams_by_size

## Learning

We will run several simulations on a generated dataset, the dataset is generated like this:
$$ \text{We have users U and actions A } u_i \sim N(0, I_{emb_dim}) \ a_i \sim N(0, I_{emb_dim})$$
$$ p_{ij} = 1 / (5 + e^{-(u_i.T a_j)}) $$
$$r_{ij} \sim Bin(p_{ij})$$

We have a policy $\pi$
and it's ground truth reward is calculated by
$$R_{gt} = \sum_{i}{\sum_{j}{\pi_{ij} * p_{ij}}} $$

Our parameters for the dataset will be
$$EmbDim = 5$$
$$NumActions= 150$$
$$NumUsers = 150$$
$$NeighborhoodSize = 6$$

to learn a new policy from $\pi$ we will sample from:
$$\pi_{start} = (1-\epsilon)*\pi + \epsilon * \pi_{random}$$

In [4]:
dataset_params = dict(
                    n_actions= 500,
                    n_users = 500,
                    emb_dim = 16,
                    # sigma = 0.1,
                    eps = 0.6, # this is the epsilon for the noise in the ground truth policy representation
                    ctr = 0.1
                    )

train_dataset = generate_dataset(dataset_params)

Random Item CTR: 0.07066414727263938
Optimal greedy CTR: 0.09999926940951757
Optimal Stochastic CTR: 0.09995326955796031
Our Initial CTR: 0.08610747363354625


In [5]:
num_runs = 1
batch_size = 200
num_neighbors = 6
n_trials_for_optuna = 20
# num_rounds_list = [500, 1000, 2000, 10000, 20000]
# num_rounds_list = [500, 1000, 2000]
num_rounds_list = [15000]


# Manually define your best parameters
best_params_to_use = {
    "lr": 0.096,  # Learning rate
    "num_epochs": 5,  # Number of training epochs
    "batch_size": 64,  # Batch size for training
    "num_neighbors": 8,  # Number of neighbors for neighborhood model
    "lr_decay": 0.85  # Learning rate decay factor
}

### 1

$$emb = 0.7 * gt + 0.3 * noise$$
$$lr = 0.005$$
$$n_{epochs} = 1$$
$$BatchSize=50$$

In [6]:
print("Value of num_rounds_list:", num_rounds_list)

# Run the optimization
df4, best_hyperparams_by_size = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, val_size=10000, n_trials=n_trials_for_optuna, prev_best_params=best_params_to_use)

# # Print best hyperparameters for each training size
# print("\n=== BEST HYPERPARAMETERS BY TRAINING SIZE ===")
# for train_size, params in best_hyperparams_by_size.items():
#     print(f"\nTraining Size: {train_size}")
#     # print(f"Best Reward: {params['reward']:.6f}")
#     print("Parameters:")
#     for param_name, value in params['params'].items():
#         print(f"  {param_name}: {value}")
# print("===========================\n")

# Show the performance metrics
df4[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

Value of num_rounds_list: [15000]
Num samples is 10000
{'gini': np.float64(0.46142294550000995), 'ess': np.float64(4503.153508554867), 'max_wi': np.float64(28.177135418152538), 'min_wi': np.float64(0.016210388551656126)}


[I 2025-10-30 08:01:27,160] A new study created in memory with name: no-name-02fa59cb-a71b-43cc-bad2-9faebea491ec
  0%|          | 0/20 [00:00<?, ?it/s]


Trial 0 started


Best trial: 0. Best value: 0.0754704:   5%|▌         | 1/20 [00:47<14:54, 47.07s/it]

Train wi info: {'gini': np.float64(0.6341531904649482), 'ess': np.float64(410.6799530834273), 'max_wi': np.float64(340.03196344205617), 'min_wi': np.float64(6.750574145678055e-07)}
actual reward: [0.07073206]
{'gini': np.float64(0.6515145781425156), 'ess': np.float64(302.832980423148), 'max_wi': np.float64(320.9408392026547), 'min_wi': np.float64(9.542057321612939e-05)}
Estimated reward: 0.084256
Cross-validated error: 0.004393
Final score CI (reward +- 2*error): [0.075470, 0.093041]
Standard error: 0.000201
Final t_dist CI (reward +- t_0.975*se_hat): [0.083862, 0.084650]
[I 2025-10-30 08:02:14,232] Trial 0 finished with value: 0.07547039259738207 and parameters: {'lr': 0.096, 'num_epochs': 5, 'batch_size': 64, 'num_neighbors': 8, 'lr_decay': 0.85}. Best is trial 0 with value: 0.07547039259738207.

Trial 1 started


...

Train wi info: {'gini': np.float64(0.15061922810595654), 'ess': np.float64(13795.065217939986), 'max_wi': np.float64(5.4136054805876945), 'min_wi': np.float64(0.00026304226079662215)}
actual reward: [0.08439124]
{'gini': np.float64(0.16954061078466437), 'ess': np.float64(9056.326186115817), 'max_wi': np.float64(4.261430472950683), 'min_wi': np.float64(0.00015663834949572545)}
Estimated reward: 0.087675
Cross-validated error: 0.001102
Final score CI (reward +- 2*error): [0.085471, 0.089880]
Standard error: 0.000175
Final t_dist CI (reward +- t_0.975*se_hat): [0.087333, 0.088018]
[I 2025-10-30 08:18:03,842] Trial 18 finished with value: 0.08547107348985081 and parameters: {'lr': 0.015215239083721398, 'num_epochs': 4, 'batch_size': 64, 'num_neighbors': 5, 'lr_decay': 0.9076484633916917}. Best is trial 11 with value: 0.08644917027680868.

Trial 19 started


Best trial: 11. Best value: 0.0864492: 100%|██████████| 20/20 [17:26<00:00, 52.32s/it]

Train wi info: {'gini': np.float64(0.0004764007305271415), 'ess': np.float64(14999.988804834464), 'max_wi': np.float64(1.004518773859392), 'min_wi': np.float64(0.9947357519976215)}
actual reward: [0.08610759]
{'gini': np.float64(0.0005610134689238554), 'ess': np.float64(9999.98978424999), 'max_wi': np.float64(1.0049159502487324), 'min_wi': np.float64(0.9950561950481522)}
Estimated reward: 0.088199
Cross-validated error: 0.001055
Final score CI (reward +- 2*error): [0.086088, 0.090309]
Standard error: 0.000160
Final t_dist CI (reward +- t_0.975*se_hat): [0.087884, 0.088513]
[I 2025-10-30 08:18:53,591] Trial 19 finished with value: 0.08608816316581142 and parameters: {'lr': 0.00033735036344289805, 'num_epochs': 7, 'batch_size': 512, 'num_neighbors': 7, 'lr_decay': 0.8347291188329855}. Best is trial 11 with value: 0.08644917027680868.


Num samples is 10000
{'gini': np.float64(0.4765721781663048), 'ess': np.float64(3904.0745315173544), 'max_wi': np.float64(39.45218623820159), 'min_wi': np.float64(0.011492300651965544)}


,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08610747,0.08500000,0.08517303,0.08716017,0.08558992,0.08558992,0.75692870,0.00000000,0.87627132,0.00000000
15000,0.08610794,0.07890156,0.08612908,0.08852331,0.08053830,0.08053830,0.75692604,0.00022642,0.87627071,0.00028259


### Policy with delta function

In [7]:
dataset_params = dict(
                    n_actions= 500,
                    n_users = 500,
                    emb_dim = 16,
                    # sigma = 0.1,
                    eps = 0.6, # this is the epsilon for the noise in the ground truth policy representation
                    ctr = 0.1
                    )

train_dataset = generate_dataset(dataset_params, seed=10000)

Random Item CTR: 0.07083863592474163
Optimal greedy CTR: 0.09999916436977967
Optimal Stochastic CTR: 0.0999493542444427
Our Initial CTR: 0.08557719469284641


In [ ]:
# Run the optimization
df5, best_hyperparams_by_size = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, val_size=10000, n_trials=n_trials_for_optuna, prev_best_params=best_params_to_use)

# Show the performance metrics
df5[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

Num samples is 10000
{'gini': np.float64(0.4576418729487834), 'ess': np.float64(4642.846415450562), 'max_wi': np.float64(16.91568930542827), 'min_wi': np.float64(0.018040019867868514)}


[I 2025-10-30 08:20:11,638] A new study created in memory with name: no-name-ac14b6f1-7d26-47d5-88ef-58b2d747ab4d
  0%|          | 0/20 [00:00<?, ?it/s]


Trial 0 started


Best trial: 0. Best value: 0.081161:   5%|▌         | 1/20 [00:40<12:44, 40.22s/it]

Train wi info: {'gini': np.float64(0.5014083231851084), 'ess': np.float64(439.2106793968737), 'max_wi': np.float64(562.5335914014425), 'min_wi': np.float64(6.01128538720611e-07)}
actual reward: [0.07255944]
{'gini': np.float64(0.496548445906111), 'ess': np.float64(601.483607384798), 'max_wi': np.float64(361.28112852388745), 'min_wi': np.float64(5.658846244252586e-07)}
Estimated reward: 0.087559
Cross-validated error: 0.003199
Final score CI (reward +- 2*error): [0.081161, 0.093957]
Standard error: 0.000161
Final t_dist CI (reward +- t_0.975*se_hat): [0.087244, 0.087874]
[I 2025-10-30 08:20:51,850] Trial 0 finished with value: 0.0811610318997957 and parameters: {'lr': 0.096, 'num_epochs': 5, 'batch_size': 64, 'num_neighbors': 8, 'lr_decay': 0.85}. Best is trial 0 with value: 0.0811610318997957.

Trial 1 started


...

Train wi info: {'gini': np.float64(0.06920052981062275), 'ess': np.float64(14752.942104945732), 'max_wi': np.float64(1.9754780971041197), 'min_wi': np.float64(0.24400514464028789)}
actual reward: [0.08539237]
{'gini': np.float64(0.06888301536393962), 'ess': np.float64(9838.970648528919), 'max_wi': np.float64(1.7803708747845193), 'min_wi': np.float64(0.32516525260814394)}
Estimated reward: 0.092955
Cross-validated error: 0.001762
Final score CI (reward +- 2*error): [0.089430, 0.096479]
Standard error: 0.000139
Final t_dist CI (reward +- t_0.975*se_hat): [0.092682, 0.093227]
[I 2025-10-30 08:30:01,955] Trial 11 finished with value: 0.08943015725688841 and parameters: {'lr': 0.013371697458452786, 'num_epochs': 2, 'batch_size': 512, 'num_neighbors': 11, 'lr_decay': 0.9898806081109537}. Best is trial 10 with value: 0.0897731173349653.

Trial 12 started


Best trial: 10. Best value: 0.0897731:  65%|██████▌   | 13/20 [10:40<05:49, 49.89s/it]

Train wi info: {'gini': np.float64(0.0551934686533476), 'ess': np.float64(14842.985121448144), 'max_wi': np.float64(1.7027764211983578), 'min_wi': np.float64(0.3163940537848398)}
actual reward: [0.08552387]
{'gini': np.float64(0.05318580687657017), 'ess': np.float64(9903.30473617357), 'max_wi': np.float64(1.586581889446515), 'min_wi': np.float64(0.3428279057901277)}
Estimated reward: 0.093350
Cross-validated error: 0.001838
Final score CI (reward +- 2*error): [0.089673, 0.097027]
Standard error: 0.000143
Final t_dist CI (reward +- t_0.975*se_hat): [0.093071, 0.093630]
[I 2025-10-30 08:30:52,482] Trial 12 finished with value: 0.08967346974590484 and parameters: {'lr': 0.011383909951347916, 'num_epochs': 1, 'batch_size': 512, 'num_neighbors': 10, 'lr_decay': 0.997639357868493}. Best is trial 10 with value: 0.0897731173349653.

Trial 13 started


Best trial: 10. Best value: 0.0897731:  70%|███████   | 14/20 [11:30<04:59, 49.87s/it]

Train wi info: {'gini': np.float64(0.06886321750341755), 'ess': np.float64(14718.219847781942), 'max_wi': np.float64(1.8331836756468007), 'min_wi': np.float64(0.09236503310655385)}
actual reward: [0.08518728]
{'gini': np.float64(0.07366917324545702), 'ess': np.float64(9785.32411847701), 'max_wi': np.float64(1.7844568138385748), 'min_wi': np.float64(0.07674054841452102)}
Estimated reward: 0.092720
Cross-validated error: 0.001728
Final score CI (reward +- 2*error): [0.089265, 0.096175]
Standard error: 0.000134
Final t_dist CI (reward +- t_0.975*se_hat): [0.092459, 0.092982]
[I 2025-10-30 08:31:42,295] Trial 13 finished with value: 0.08926512157554124 and parameters: {'lr': 0.01385946345030374, 'num_epochs': 6, 'batch_size': 512, 'num_neighbors': 12, 'lr_decay': 0.9629408227963585}. Best is trial 10 with value: 0.0897731173349653.

Trial 14 started


In [ ]:
dataset_params = dict(
                    n_actions= 500,
                    n_users = 500,
                    emb_dim = 16,
                    # sigma = 0.1,
                    eps = 0.6, # this is the epsilon for the noise in the ground truth policy representation
                    ctr = 0.1
                    )

train_dataset = generate_dataset(dataset_params, seed=20000)

Random Item CTR: 0.07042251854546815
Optimal greedy CTR: 0.09999934264692525
Optimal Stochastic CTR: 0.09996075464321043
Our Initial CTR: 0.08647580588501355


In [ ]:
# Run the optimization
df6, best_hyperparams_by_size = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, val_size=10000, n_trials=n_trials_for_optuna, prev_best_params=best_params_to_use)

# Show the performance metrics
df6[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

Num samples is 10000
{'gini': np.float64(0.48022945838555753), 'ess': np.float64(3954.734881758623), 'max_wi': np.float64(38.92603878947978), 'min_wi': np.float64(0.006564213856433456)}


[I 2025-10-29 22:56:19,596] A new study created in memory with name: no-name-8ec402b9-876c-4bb8-a66d-4bd6797153dc
  0%|          | 0/20 [00:00<?, ?it/s]


Trial 0 started


Best trial: 0. Best value: 0.0789441:   5%|▌         | 1/20 [00:44<14:13, 44.93s/it]

Train wi info: {'gini': np.float64(0.5647763877991416), 'ess': np.float64(519.1888176636096), 'max_wi': np.float64(457.9536865355126), 'min_wi': np.float64(2.0446657031422895e-07)}
actual reward: [0.07126326]
{'gini': np.float64(0.5394520397752793), 'ess': np.float64(2120.742059473937), 'max_wi': np.float64(65.98318679436115), 'min_wi': np.float64(1.1044849972608172e-05)}
Estimated reward: 0.085064
Cross-validated error: 0.003060
Final score CI (reward +- 2*error): [0.078944, 0.091184]
Standard error: 0.000440
Final t_dist CI (reward +- t_0.975*se_hat): [0.084201, 0.085927]
[I 2025-10-29 22:57:04,519] Trial 0 finished with value: 0.07894412984147801 and parameters: {'lr': 0.096, 'num_epochs': 5, 'batch_size': 64, 'num_neighbors': 8, 'lr_decay': 0.85}. Best is trial 0 with value: 0.07894412984147801.

Trial 1 started


...

Train wi info: {'gini': np.float64(0.0020114217834353108), 'ess': np.float64(14999.796199714254), 'max_wi': np.float64(1.030928651045261), 'min_wi': np.float64(0.9770325120085369)}
actual reward: [0.08647747]
{'gini': np.float64(0.0027865879534815434), 'ess': np.float64(9999.741067860808), 'max_wi': np.float64(1.028284917299221), 'min_wi': np.float64(0.9728758626315079)}
Estimated reward: 0.088517
Cross-validated error: 0.001139
Final score CI (reward +- 2*error): [0.086239, 0.090796]
Standard error: 0.000380
Final t_dist CI (reward +- t_0.975*se_hat): [0.087773, 0.089262]
[I 2025-10-29 23:12:51,956] Trial 18 finished with value: 0.08623857696289441 and parameters: {'lr': 0.0006867280325907604, 'num_epochs': 8, 'batch_size': 64, 'num_neighbors': 12, 'lr_decay': 0.8288813969065628}. Best is trial 11 with value: 0.08748258253041329.

Trial 19 started


Best trial: 11. Best value: 0.0874826: 100%|██████████| 20/20 [17:23<00:00, 52.18s/it]

Train wi info: {'gini': np.float64(0.04342838404880459), 'ess': np.float64(14905.348983379758), 'max_wi': np.float64(1.5194426043234959), 'min_wi': np.float64(0.299840195886946)}
actual reward: [0.08619026]
{'gini': np.float64(0.04977063774797568), 'ess': np.float64(9918.953528445223), 'max_wi': np.float64(1.5295831446853847), 'min_wi': np.float64(0.3173229878401552)}
Estimated reward: 0.089324
Cross-validated error: 0.001258
Final score CI (reward +- 2*error): [0.086808, 0.091840]
Standard error: 0.000466
Final t_dist CI (reward +- t_0.975*se_hat): [0.088410, 0.090237]
[I 2025-10-29 23:13:43,233] Trial 19 finished with value: 0.08680810788897474 and parameters: {'lr': 0.006233937728785646, 'num_epochs': 9, 'batch_size': 128, 'num_neighbors': 4, 'lr_decay': 0.9801058508596758}. Best is trial 11 with value: 0.08748258253041329.


Num samples is 10000
{'gini': np.float64(0.4652533554243185), 'ess': np.float64(3926.853063685224), 'max_wi': np.float64(51.029251933606915), 'min_wi': np.float64(0.005437645079418442)}


,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08647581,0.08770000,0.08769244,0.09067430,0.08747803,0.08747803,0.88083979,0.00000000,0.74725465,0.00000000
15000,0.08637521,0.07921840,0.08685918,0.08963190,0.07886415,0.07886773,0.87758895,0.02348538,0.74505549,0.02166221


In [ ]:
dataset_params = dict(
                    n_actions= 500,
                    n_users = 500,
                    emb_dim = 16,
                    # sigma = 0.1,
                    eps = 0.6, # this is the epsilon for the noise in the ground truth policy representation
                    ctr = 0.1
                    )

train_dataset = generate_dataset(dataset_params, seed=30000)

Random Item CTR: 0.07069350185865088
Optimal greedy CTR: 0.09999918303816259
Optimal Stochastic CTR: 0.0999509448932121
Our Initial CTR: 0.08653966603258505


In [ ]:
# Run the optimization
df7, best_hyperparams_by_size = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, val_size=10000, n_trials=n_trials_for_optuna, prev_best_params=best_params_to_use)

# Show the performance metrics
df7[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

Num samples is 10000
{'gini': np.float64(0.4791613368557878), 'ess': np.float64(4347.254707099757), 'max_wi': np.float64(20.842543447049458), 'min_wi': np.float64(0.0053891659002369445)}


[I 2025-10-29 23:15:02,697] A new study created in memory with name: no-name-d2526723-dbdd-40a7-bcd1-db2f404e8992
  0%|          | 0/20 [00:00<?, ?it/s]


Trial 0 started


Best trial: 0. Best value: 0.0784675:   5%|▌         | 1/20 [00:40<12:44, 40.24s/it]

Train wi info: {'gini': np.float64(0.5140795775215294), 'ess': np.float64(2439.845254714719), 'max_wi': np.float64(116.05822770095907), 'min_wi': np.float64(2.4167185259894426e-07)}
actual reward: [0.07062138]
{'gini': np.float64(0.5410011860369925), 'ess': np.float64(1018.7810412118494), 'max_wi': np.float64(185.3093249170202), 'min_wi': np.float64(4.2872526867514565e-08)}
Estimated reward: 0.084684
Cross-validated error: 0.003108
Final score CI (reward +- 2*error): [0.078468, 0.090901]
Standard error: 0.000439
Final t_dist CI (reward +- t_0.975*se_hat): [0.083824, 0.085544]
[I 2025-10-29 23:15:42,939] Trial 0 finished with value: 0.07846753922494211 and parameters: {'lr': 0.096, 'num_epochs': 5, 'batch_size': 64, 'num_neighbors': 8, 'lr_decay': 0.85}. Best is trial 0 with value: 0.07846753922494211.

Trial 1 started


...

Train wi info: {'gini': np.float64(0.004258687408258315), 'ess': np.float64(14999.102603395944), 'max_wi': np.float64(1.0404459710137208), 'min_wi': np.float64(0.9645295059352461)}
actual reward: [0.08653811]
{'gini': np.float64(0.004284587532293961), 'ess': np.float64(9999.408803196207), 'max_wi': np.float64(1.0325169162414054), 'min_wi': np.float64(0.9642340017043597)}
Estimated reward: 0.088968
Cross-validated error: 0.001325
Final score CI (reward +- 2*error): [0.086318, 0.091618]
Standard error: 0.000411
Final t_dist CI (reward +- t_0.975*se_hat): [0.088163, 0.089773]
[I 2025-10-29 23:30:49,728] Trial 18 finished with value: 0.08631800079160452 and parameters: {'lr': 0.0015420045958438756, 'num_epochs': 4, 'batch_size': 512, 'num_neighbors': 11, 'lr_decay': 0.874724814856287}. Best is trial 9 with value: 0.0864701818888323.

Trial 19 started


Best trial: 9. Best value: 0.0864702: 100%|██████████| 20/20 [16:37<00:00, 49.89s/it]

Train wi info: {'gini': np.float64(0.0038997351414210984), 'ess': np.float64(14999.254789295452), 'max_wi': np.float64(1.043412333875427), 'min_wi': np.float64(0.9645372290810577)}
actual reward: [0.08653813]
{'gini': np.float64(0.003776951437468392), 'ess': np.float64(9999.543448183622), 'max_wi': np.float64(1.0338350171542912), 'min_wi': np.float64(0.9706413167283172)}
Estimated reward: 0.088738
Cross-validated error: 0.001460
Final score CI (reward +- 2*error): [0.085818, 0.091659]
Standard error: 0.000436
Final t_dist CI (reward +- t_0.975*se_hat): [0.087883, 0.089594]
[I 2025-10-29 23:31:40,550] Trial 19 finished with value: 0.08581828432786082 and parameters: {'lr': 0.00041123574827441777, 'num_epochs': 2, 'batch_size': 64, 'num_neighbors': 8, 'lr_decay': 0.915231414544563}. Best is trial 9 with value: 0.0864701818888323.


Num samples is 10000
{'gini': np.float64(0.47317254545286797), 'ess': np.float64(4208.429618140219), 'max_wi': np.float64(26.142866325336392), 'min_wi': np.float64(0.0062482056410791366)}


,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08653967,0.08660000,0.08656501,0.08669117,0.08551839,0.08551839,0.82469903,0.00000000,0.72168239,0.00000000
15000,0.08654042,0.07710110,0.08779074,0.08891530,0.07702781,0.07702758,0.82468488,0.00034299,0.72167100,0.00045910


In [ ]:
dataset_params = dict(
                    n_actions= 500,
                    n_users = 500,
                    emb_dim = 16,
                    # sigma = 0.1,
                    eps = 0.6, # this is the epsilon for the noise in the ground truth policy representation
                    ctr = 0.1
                    )

train_dataset = generate_dataset(dataset_params, seed=40000)

Random Item CTR: 0.07053370144999074
Optimal greedy CTR: 0.09999936716169436
Optimal Stochastic CTR: 0.09995563088920843
Our Initial CTR: 0.08622184481781218


In [ ]:
# Run the optimization
df8, best_hyperparams_by_size = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, val_size=10000, n_trials=n_trials_for_optuna, prev_best_params=best_params_to_use)

# Show the performance metrics
df8[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

Num samples is 10000
{'gini': np.float64(0.4619849198177038), 'ess': np.float64(4469.8281246147635), 'max_wi': np.float64(24.209173244022438), 'min_wi': np.float64(0.011780621871838282)}


[I 2025-10-29 23:32:51,382] A new study created in memory with name: no-name-a308de91-277c-443a-b6f3-741a57e32284
  0%|          | 0/20 [00:00<?, ?it/s]


Trial 0 started


Best trial: 0. Best value: 0.0766376:   5%|▌         | 1/20 [00:40<12:44, 40.24s/it]

Train wi info: {'gini': np.float64(0.5978836988482632), 'ess': np.float64(584.1645131553638), 'max_wi': np.float64(254.72112163748818), 'min_wi': np.float64(5.413791320068287e-08)}
actual reward: [0.07073242]
{'gini': np.float64(0.6326912024315681), 'ess': np.float64(361.12048268412855), 'max_wi': np.float64(275.55849406163674), 'min_wi': np.float64(4.4788291658694265e-05)}
Estimated reward: 0.084581
Cross-validated error: 0.003972
Final score CI (reward +- 2*error): [0.076638, 0.092524]
Standard error: 0.000374
Final t_dist CI (reward +- t_0.975*se_hat): [0.083848, 0.085314]
[I 2025-10-29 23:33:31,616] Trial 0 finished with value: 0.0766376312209124 and parameters: {'lr': 0.096, 'num_epochs': 5, 'batch_size': 64, 'num_neighbors': 8, 'lr_decay': 0.85}. Best is trial 0 with value: 0.0766376312209124.

Trial 1 started


...

Train wi info: {'gini': np.float64(0.013976063662075258), 'ess': np.float64(14990.330270145507), 'max_wi': np.float64(1.143844832302929), 'min_wi': np.float64(0.8624492438853154)}
actual reward: [0.08621685]
{'gini': np.float64(0.014054671459171383), 'ess': np.float64(9993.611044745907), 'max_wi': np.float64(1.1136849109643847), 'min_wi': np.float64(0.8947909210474199)}
Estimated reward: 0.088821
Cross-validated error: 0.001455
Final score CI (reward +- 2*error): [0.085911, 0.091731]
Standard error: 0.000323
Final t_dist CI (reward +- t_0.975*se_hat): [0.088188, 0.089454]
[I 2025-10-29 23:48:37,240] Trial 18 finished with value: 0.0859105206622413 and parameters: {'lr': 0.0019558798762453027, 'num_epochs': 3, 'batch_size': 128, 'num_neighbors': 9, 'lr_decay': 0.9183017950207495}. Best is trial 13 with value: 0.08656017892556978.

Trial 19 started


Best trial: 13. Best value: 0.0865602: 100%|██████████| 20/20 [16:36<00:00, 49.85s/it]

Train wi info: {'gini': np.float64(0.0010793670251309823), 'ess': np.float64(14999.942065276055), 'max_wi': np.float64(1.0114085071882766), 'min_wi': np.float64(0.9864329664383559)}
actual reward: [0.08622249]
{'gini': np.float64(0.0010930030041018546), 'ess': np.float64(9999.960816091478), 'max_wi': np.float64(1.010312908777103), 'min_wi': np.float64(0.9876586393756517)}
Estimated reward: 0.089102
Cross-validated error: 0.001365
Final score CI (reward +- 2*error): [0.086372, 0.091832]
Standard error: 0.000342
Final t_dist CI (reward +- t_0.975*se_hat): [0.088432, 0.089772]
[I 2025-10-29 23:49:28,325] Trial 19 finished with value: 0.08637199034576998 and parameters: {'lr': 0.00029571771250444057, 'num_epochs': 7, 'batch_size': 256, 'num_neighbors': 7, 'lr_decay': 0.9732130603730548}. Best is trial 13 with value: 0.08656017892556978.


Num samples is 10000
{'gini': np.float64(0.4672203367029539), 'ess': np.float64(4337.54526331691), 'max_wi': np.float64(31.43189456680396), 'min_wi': np.float64(0.019415905542555838)}


,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08622184,0.08840000,0.08855226,0.09106993,0.08852082,0.08852082,0.92210476,0.00000000,0.83772226,0.00000000
15000,0.08622071,0.07708458,0.08807528,0.08898085,0.07576440,0.07576369,0.92197805,0.00253678,0.83771236,0.00303496


In [ ]:
dataset_params = dict(
                    n_actions= 500,
                    n_users = 500,
                    emb_dim = 16,
                    # sigma = 0.1,
                    eps = 0.6, # this is the epsilon for the noise in the ground truth policy representation
                    ctr = 0.1
                    )

train_dataset = generate_dataset(dataset_params, seed=50000)

Random Item CTR: 0.0705882181025533
Optimal greedy CTR: 0.09999934164533562
Optimal Stochastic CTR: 0.09995498601895662
Our Initial CTR: 0.08647501952799874


In [ ]:
# Run the optimization
df9, best_hyperparams_by_size = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, val_size=10000, n_trials=n_trials_for_optuna, prev_best_params=best_params_to_use)

# Show the performance metrics
df9[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

Num samples is 10000
{'gini': np.float64(0.49847821557978683), 'ess': np.float64(3945.2567055809536), 'max_wi': np.float64(32.439454875391334), 'min_wi': np.float64(0.006670832088175685)}


[I 2025-10-29 23:50:46,492] A new study created in memory with name: no-name-4c07dae2-9632-4353-85aa-76944ab38e61
  0%|          | 0/20 [00:00<?, ?it/s]


Trial 0 started


Best trial: 0. Best value: 0.0803135:   5%|▌         | 1/20 [00:41<13:16, 41.90s/it]

Train wi info: {'gini': np.float64(0.5405386868433234), 'ess': np.float64(471.9726311635205), 'max_wi': np.float64(394.39940460067965), 'min_wi': np.float64(1.1884447622076383e-07)}
actual reward: [0.07110314]
{'gini': np.float64(0.6001936644532129), 'ess': np.float64(297.5981305282253), 'max_wi': np.float64(266.3065819300526), 'min_wi': np.float64(2.499720611856428e-06)}
Estimated reward: 0.087618
Cross-validated error: 0.003652
Final score CI (reward +- 2*error): [0.080313, 0.094923]
Standard error: 0.000298
Final t_dist CI (reward +- t_0.975*se_hat): [0.087034, 0.088202]
[I 2025-10-29 23:51:28,391] Trial 0 finished with value: 0.08031347792721162 and parameters: {'lr': 0.096, 'num_epochs': 5, 'batch_size': 64, 'num_neighbors': 8, 'lr_decay': 0.85}. Best is trial 0 with value: 0.08031347792721162.

Trial 1 started


...

Train wi info: {'gini': np.float64(0.004242358079451612), 'ess': np.float64(14999.115156063288), 'max_wi': np.float64(1.0350545581678174), 'min_wi': np.float64(0.9560888271347018)}
actual reward: [0.08647202]
{'gini': np.float64(0.004179398382807423), 'ess': np.float64(9999.434529961114), 'max_wi': np.float64(1.0323227147710987), 'min_wi': np.float64(0.9656071095239178)}
Estimated reward: 0.089230
Cross-validated error: 0.001276
Final score CI (reward +- 2*error): [0.086678, 0.091781]
Standard error: 0.000317
Final t_dist CI (reward +- t_0.975*se_hat): [0.088608, 0.089852]
[I 2025-10-30 00:06:24,934] Trial 18 finished with value: 0.0866781603445874 and parameters: {'lr': 0.001211098479460515, 'num_epochs': 3, 'batch_size': 512, 'num_neighbors': 4, 'lr_decay': 0.9354736847647402}. Best is trial 11 with value: 0.0872957294752143.

Trial 19 started


Best trial: 11. Best value: 0.0872957: 100%|██████████| 20/20 [16:28<00:00, 49.44s/it]

Train wi info: {'gini': np.float64(0.026892551826398162), 'ess': np.float64(14963.437547063657), 'max_wi': np.float64(1.3463478538445457), 'min_wi': np.float64(0.643063360098534)}
actual reward: [0.08640608]
{'gini': np.float64(0.028284390029761717), 'ess': np.float64(9973.311320912575), 'max_wi': np.float64(1.2776541469697276), 'min_wi': np.float64(0.643063360098534)}
Estimated reward: 0.088900
Cross-validated error: 0.001192
Final score CI (reward +- 2*error): [0.086515, 0.091285]
Standard error: 0.000257
Final t_dist CI (reward +- t_0.975*se_hat): [0.088396, 0.089404]
[I 2025-10-30 00:07:15,368] Trial 19 finished with value: 0.08651518692572786 and parameters: {'lr': 0.003224914608873256, 'num_epochs': 7, 'batch_size': 128, 'num_neighbors': 11, 'lr_decay': 0.996574043659471}. Best is trial 11 with value: 0.0872957294752143.


Num samples is 10000
{'gini': np.float64(0.4940130647098055), 'ess': np.float64(2945.110828404994), 'max_wi': np.float64(95.5346006507542), 'min_wi': np.float64(0.008504698012894578)}


,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08647502,0.08760000,0.08752670,0.09086329,0.08530194,0.08530194,0.80232812,0.00000000,0.84032376,0.00000000
15000,0.08647562,0.07880134,0.08684865,0.08955903,0.08058818,0.08058820,0.80232088,0.00009224,0.84031447,0.00013923


In [ ]:
df4[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08610747,0.08600000,0.08594797,0.08791196,0.08599389,0.08599389,0.75692870,0.00000000,0.87627132,0.00000000
15000,0.08610291,0.07783040,0.08748281,0.09035353,0.07790566,0.07790813,0.75684444,0.00513398,0.87628721,0.00745030


### Poicy Via argmax(r_hat - error_hat) through cross validation

In [ ]:
df4[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08610747,0.08600000,0.08594797,0.08791196,0.08599389,0.08599389,0.75692870,0.00000000,0.87627132,0.00000000
15000,0.08610291,0.07783040,0.08748281,0.09035353,0.07790566,0.07790813,0.75684444,0.00513398,0.87628721,0.00745030


### Policy Via using actual policy value

In [ ]:
# Show the performance metrics
df4[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]


,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08610747,0.08600000,0.08594797,0.08791196,0.08599389,0.08599389,0.75692870,0.00000000,0.87627132,0.00000000
15000,0.08610291,0.07783040,0.08748281,0.09035353,0.07790566,0.07790813,0.75684444,0.00513398,0.87628721,0.00745030
